In [2]:
import pandas as pd
import numpy as np
import seaborn as sns

pd.set_option("display.max_columns",None)
pd.set_option("display.width",500)
pd.set_option("display.float_format", lambda x: '%.2f' % x)

### 1- miuul_gezinomi.xlsx dosyasını okutunuz ve veri seti ile ilgili genel bilgileri gösteriniz..

In [3]:
df = pd.read_excel("../dataset/miuul_gezinomi.xlsx")
pd.DataFrame.set_index(df,"SaleId", inplace=True)

def check_df(dataframe, head=5):
    print("###Shape###")
    print(dataframe.shape)
    print("###Types###")
    print(dataframe.dtypes)
    print("###Head###")
    print(dataframe.head(head))
    print("###Tail###")
    print(dataframe.tail(head))
    print("###NA###")
    print(dataframe.isnull().sum())
    print("###Quantiles###")
    print(dataframe.describe([0,0.05,0.50,0.95,0.99,1]).T)

check_df(df)

###Shape###
(59164, 8)
###Types###
SaleDate              datetime64[ns]
CheckInDate           datetime64[ns]
Price                        float64
ConceptName                   object
SaleCityName                  object
CInDay                        object
SaleCheckInDayDiff             int64
Seasons                       object
dtype: object
###Head###
         SaleDate CheckInDate  Price     ConceptName SaleCityName    CInDay  SaleCheckInDayDiff Seasons
SaleId                                                                                                 
415122 2022-12-03  2022-12-03  79.30    Herşey Dahil      Antalya  Saturday                   0     Low
415103 2022-12-03  2022-12-03  45.97  Yarım Pansiyon      Antalya  Saturday                   0     Low
404034 2022-09-12  2022-09-13  77.84    Herşey Dahil      Antalya   Tuesday                   1    High
415094 2022-12-03  2022-12-10 222.71  Yarım Pansiyon        İzmir  Saturday                   7     Low
414951 2022-12-01  2

### 2- Kaç unique şehir vardır? Frekansları nedir?

In [15]:
df["SaleCityName"].nunique()

6

In [16]:
df["SaleCityName"].value_counts()

Antalya    31649
Muğla      10662
Aydın      10646
Diğer       3245
İzmir       2507
Girne        455
Name: SaleCityName, dtype: int64

### 3- Kaç unique Concept vardır?

In [17]:
df["ConceptName"].nunique()

3

### 4- Hangi Concept’den kaçar tane satış gerçekleşmiş?

In [18]:
df["ConceptName"].value_counts()

Herşey Dahil      53186
Yarım Pansiyon     3559
Oda + Kahvaltı     2419
Name: ConceptName, dtype: int64

### 5- Şehirlere göre satışlardan toplam ne kadar kazanılmış?

In [24]:
#df.groupby("SaleCityName")["Price"].sum()
df.groupby("SaleCityName").agg({"Price": "sum"})

,Price
SaleCityName,
Antalya,2041911.10
Aydın,573296.01
Diğer,154572.29
Girne,27065.03
Muğla,665842.21
İzmir,165934.83


### 6- Concept türlerine göre göre ne kadar kazanılmış?

In [25]:
#df.groupby("ConceptName")["Price"].sum()
df.groupby("ConceptName").agg({"Price": "sum"})

,Price
ConceptName,
Herşey Dahil,3332910.77
Oda + Kahvaltı,121308.35
Yarım Pansiyon,174402.35


### 7- Şehirlere göre PRICE ortalamaları nedir?

In [26]:
#df.groupby("SaleCityName")["Price"].mean()
df.groupby("SaleCityName").agg({"Price": "mean"})

,Price
SaleCityName,
Antalya,64.52
Aydın,53.86
Diğer,47.71
Girne,59.48
Muğla,62.46
İzmir,66.27


### 8- Conceptlere göre PRICE ortalamaları nedir?

In [27]:
#df.groupby("ConceptName")["Price"].mean()
df.groupby("ConceptName").agg({"Price": "mean"})

,Price
ConceptName,
Herşey Dahil,62.67
Oda + Kahvaltı,50.25
Yarım Pansiyon,49.03


### 9- Şehir-Concept kırılımında PRICE ortalamaları nedir?

In [28]:
#df.groupby(["SaleCityName","ConceptName"])["Price"].mean()
df.groupby(["SaleCityName","ConceptName"]).agg({"Price": "mean"})

Price
SaleCityName ConceptName          
Antalya      Herşey Dahil    64.52
             Oda + Kahvaltı  63.50
             Yarım Pansiyon  67.19
Aydın        Herşey Dahil    54.00
             Oda + Kahvaltı  34.46
             Yarım Pansiyon  30.02
Diğer        Herşey Dahil    84.77
             Oda + Kahvaltı  37.60
             Yarım Pansiyon  42.11
Girne        Herşey Dahil    97.68
             Oda + Kahvaltı  39.78
             Yarım Pansiyon  53.25
Muğla        Herşey Dahil    63.02
             Oda + Kahvaltı  59.04
             Yarım Pansiyon  45.12
İzmir        Herşey Dahil    74.70
             Oda + Kahvaltı  41.32
             Yarım Pansiyon  59.61

# Task 2

### SaleCheckInDayDiff değişkeni müşterinin CheckIn tarihinden ne kadar önce satin alımını tamamladığını gösterir.
### Aralıkları ikna edici şekilde oluşturunuz.
### Örneğin: ‘0_7’, ‘7_30', ‘30_90', ‘90_max’ aralıklarını kullanabilirsiniz.
### Bu aralıklar için "Last Minuters", "Potential Planners", "Planners", "Early Bookers“ isimlerini kullanabilirsiniz.

In [24]:
df["SaleCheckInDayDiff"]


SaleId
415122     0 days
415103     0 days
404034     1 days
415094     7 days
414951     2 days
           ...   
51817    279 days
51816    279 days
51814      1 days
51736      1 days
51731    231 days
Name: SaleCheckInDayDiff, Length: 59164, dtype: timedelta64[ns]

In [38]:
bins = [-1,7,30,90, df["SaleCheckInDayDiff"].max()]
bins = pd.to_timedelta(bins)
categories = ["Last Minute", "Potential Planners", "Planners", "Early Bookers"]

df["EB_score"] = pd.cut(df["SaleCheckInDayDiff"], bins, labels= categories)
df.head(50).to_excel("eb_score.xlsx", index=False)


In [39]:
bins

TimedeltaIndex(['-1 days +23:59:59.999999999', '0 days 00:00:00.000000007', '0 days 00:00:00.000000030', '0 days 00:00:00.000000090', '630 days 00:00:00'], dtype='timedelta64[ns]', freq=None)